# **Chapter 12: The Cloud Shared Responsibility Model and Security Architecture**

## Introduction: The Security Paradigm Shift

The migration to cloud computing fundamentally transforms how organizations approach security. In traditional on-premises data centers, the organization owned the entire security stack—from the physical data center perimeter to the application layer. Cloud computing introduces a shared responsibility model where security obligations are distributed between the cloud provider and the customer. Understanding this boundary is not merely an academic exercise; misconfiguration and misunderstanding of responsibility boundaries remain the leading causes of cloud security breaches.

This chapter establishes the foundational security architecture for cloud environments. We will dissect the shared responsibility model across service models (IaaS, PaaS, SaaS), implement Zero Trust architectures that assume breach, establish data protection strategies spanning encryption at rest and in transit, and design network security controls for multi-cloud environments. These principles provide the security foundation upon which the identity management, infrastructure security, and operational security practices in subsequent chapters are built.

---

## 12.1 Understanding the Shared Responsibility Model

The Shared Responsibility Model is the contractual and operational framework defining which security tasks are handled by the cloud provider and which remain the customer's obligation. This model shifts based on the service model employed—IaaS places more responsibility on the customer, while SaaS transfers most security obligations to the provider.

### 12.1.1 The Responsibility Spectrum

**Infrastructure as a Service (IaaS):**
- **Provider Responsibility:** Physical security of data centers, hardware lifecycle management, network infrastructure (routers, switches), hypervisor security
- **Customer Responsibility:** Operating system patching, network configuration (security groups, NACLs), application security, data encryption, identity management, traffic protection
- **Shared:** Network firewall configuration (provider offers capability; customer defines rules), compliance certifications (provider maintains infrastructure certifications; customer maintains application-level compliance)

**Platform as a Service (PaaS):**
- **Provider Responsibility:** Operating system patching, middleware management, runtime environments, underlying infrastructure
- **Customer Responsibility:** Application code security, data classification and encryption, identity and access management (IAM) configuration, API security
- **Shared:** Network security policies, logging and monitoring configuration, compliance validation

**Software as a Service (SaaS):**
- **Provider Responsibility:** Nearly all technical security—application security, data storage security, physical and infrastructure layers
- **Customer Responsibility:** Data classification, user access management (provisioning/deprovisioning), endpoint device security, ensuring compliant data usage
- **Shared:** Data residency requirements, encryption key management (depending on offering), audit logging access

### 12.1.2 Visualizing the Model

```
On-Premises        IaaS               PaaS               SaaS
[Customer]         [Provider]         [Provider]         [Provider]
| Applications     | Applications     | Applications     | Applications
| Data             | Data             | Data             | Data
| Runtime          | Runtime          | Runtime          | Runtime
| Middleware       | Middleware       | Middleware       | Middleware
| O/S              | O/S              | O/S              | O/S
| Virtualization   | Virtualization   | Virtualization   | Virtualization
| Servers          | Servers          | Servers          | Servers
| Storage          | Storage          | Storage          | Storage
| Networking       | Networking       | Networking       | Networking
| Physical         | Physical         | Physical         | Physical
[Customer]         [Provider]         [Provider]         [Provider]
```

**Critical Insight:** As you move right (toward SaaS), you transfer operational burden but retain accountability. Even in SaaS, the customer remains responsible for data governance, user access decisions, and ensuring the provider's certifications align with regulatory requirements.

### 12.1.3 Common Misconfigurations and Breaches

**The Capital One Breach (2019):**
A classic example of IaaS responsibility misunderstanding. The attacker exploited a misconfigured Web Application Firewall (WAF)—specifically, a customer-managed security group that allowed SSRF (Server-Side Request Forgery) attacks. The cloud provider (AWS) secured the infrastructure, but the customer misconfigured the network layer, violating the shared responsibility boundary.

**Lesson:** Even with secure underlying infrastructure, customer-managed configurations (security groups, IAM policies, encryption settings) remain the primary attack vector.

**Terraform Policy as Code for Responsibility Enforcement:**

```hcl
# Sentinel policy for HashiCorp Enterprise to enforce security responsibilities
# Ensures all S3 buckets have encryption (Customer Responsibility)

import "tfplan"

# Get all S3 buckets from the plan
all_s3_buckets = filter tfplan.resource_changes as _, rc {
    rc.type is "aws_s3_bucket" and
    (rc.change.actions contains "create" or rc.change.actions contains "update")
}

# Rule: All S3 buckets must have server-side encryption configured
rule_encryption_enabled {
    all all_s3_buckets as _, bucket {
        bucket.change.after.server_side_encryption_configuration is not null
    }
}

# Rule: Public access must be explicitly configured (not left default)
rule_public_access_blocked {
    all all_s3_buckets as _, bucket {
        bucket.change.after.block_public_access_configuration is not null
    }
}

# Main rule enforcement
main = rule {
    rule_encryption_enabled and
    rule_public_access_blocked
}

# Violation messages
violations = filter all_s3_buckets as _, bucket {
    bucket.change.after.server_side_encryption_configuration is null or
    bucket.change.after.block_public_access_configuration is null
}

if length(violations) > 0 {
    print("VIOLATION: S3 buckets must have encryption and public access blocks configured")
    print("Buckets violating policy:", violations)
}
```

**Explanation:**
- **Policy as Code:** Embeds security responsibilities into the CI/CD pipeline before resources are provisioned
- **Default Deny:** Prevents deployment of resources that don't meet encryption requirements
- **Audit Trail:** Maintains evidence of security controls for compliance audits

---

## 12.2 Zero Trust Architecture in the Cloud

Zero Trust is a security paradigm that eliminates implicit trust based on network location. Instead of assuming that traffic inside the corporate network is safe, Zero Trust requires continuous verification of every user, device, and transaction, regardless of origin.

### 12.2.1 Core Principles of Zero Trust

1. **Never Trust, Always Verify:** Authenticate and authorize every access request based on all available data points (identity, device health, location, data classification)
2. **Assume Breach:** Operate as if the network is already compromised; minimize blast radius through segmentation
3. **Least Privilege Access:** Grant only the minimum permissions required for the specific context (just-in-time and just-enough access)
4. **Verify Explicitly:** Use multiple signals (MFA, device compliance, risk detection) before granting access

### 12.2.2 Implementing Zero Trust Network Access (ZTNA)

Traditional VPNs provide broad network access once authenticated. ZTNA provides granular, application-level access without exposing the underlying network.

**Architecture Components:**

**Identity-Aware Proxy (IAP):**
Sits between users and applications, verifying identity and context before allowing connections. Cloud providers offer managed IAP services (Google Cloud IAP, Azure AD Application Proxy, AWS Verified Access).

**Micro-Segmentation:**
Divides the network into small, isolated zones. In cloud environments, this means:
- Application-specific security groups (not broad "web tier" or "database tier" groups)
- Service mesh mTLS (as discussed in Chapter 11) for internal service communication
- Identity-based firewall rules rather than IP-based rules

**Example: Zero Trust Network Policy with Cilium (eBPF-based Kubernetes Networking):**

```yaml
apiVersion: cilium.io/v2
kind: CiliumNetworkPolicy
metadata:
  name: zero-trust-payment-service
  namespace: production
spec:
  endpointSelector:
    matchLabels:
      app: payment-service
  ingress:
    # Only allow traffic from specific frontend pods with specific labels
    - fromEndpoints:
        - matchLabels:
            app: frontend
            version: v2
            io.kubernetes.pod.namespace: production
      toPorts:
        - ports:
            - port: "8080"
              protocol: TCP
          rules:
            http:
              - method: POST
                path: "/api/v1/payments"
                headers:
                  - name: X-Request-ID
                    required: true
              - method: GET
                path: "/api/v1/payments/{id}/status"
    # Allow monitoring tools (specific service account)
    - fromEndpoints:
        - matchLabels:
            app: prometheus
            io.kubernetes.pod.namespace: monitoring
      toPorts:
        - ports:
            - port: "9090"
              protocol: TCP
  egress:
    # Only allow outbound to specific database with specific protocol
    - toEndpoints:
        - matchLabels:
            app: postgres
            tier: database
      toPorts:
        - ports:
            - port: "5432"
              protocol: TCP
          rules:
            # L7 filtering: Only allow specific SQL operations
            postgres:
              - action: SELECT
                table: payments
              - action: INSERT
                table: payments
              - action: UPDATE
                table: payments
    # Allow DNS resolution only to specific DNS server
    - toEndpoints:
        - matchLabels:
            k8s:io.kubernetes.pod.namespace: kube-system
            k8s:k8s-app: kube-dns
      toPorts:
        - ports:
            - port: "53"
              protocol: UDP
          rules:
            dns:
              - matchPattern: "*.postgres.production.svc.cluster.local"
```

**Zero Trust Elements:**
- **Identity-Based Rules:** Policies reference workload identity (labels, service accounts) rather than IP addresses, which change dynamically in cloud environments
- **L7 Protocol Awareness:** Inspects application-layer protocols (HTTP paths, SQL commands) rather than just allowing port 5432 traffic to any destination
- **Default Deny:** Implicit deny-all unless explicitly allowed (least privilege)

### 12.2.3 Device Trust and Context-Aware Access

Zero Trust extends beyond user identity to device posture. A managed corporate laptop with encryption and AV differs significantly from a personal tablet accessing the same application.

**Implementation with Azure AD Conditional Access / AWS Verified Access:**

```json
// Azure AD Conditional Access Policy (JSON representation)
{
    "displayName": "Zero Trust - Require Compliant Device for Sensitive Apps",
    "state": "enabled",
    "conditions": {
        "applications": {
            "includeApplications": [
                "app-id-for-sensitive-crm",
                "app-id-for-financial-data"
            ]
        },
        "users": {
            "includeGroups": ["all-employees"],
            "excludeGroups": ["emergency-access-break-glass"]
        },
        "platforms": {
            "includePlatforms": ["all"]
        },
        "locations": {
            "includeLocations": ["Any"]
        },
        "riskLevels": {
            "riskLevels": ["low", "medium"]
        }
    },
    "grantControls": {
        "operator": "AND",
        "builtInControls": [
            "mfa",
            "compliantDevice",
            "domainJoined"
        ],
        "customAuthenticationFactors": [],
        "termsOfUse": []
    },
    "sessionControls": {
        "applicationEnforcedRestrictions": null,
        "cloudAppSecurity": {
            "cloudAppSecurityType": "monitorOnly",
            "isEnabled": true
        },
        "signInFrequency": {
            "value": 4,
            "type": "hours",
            "isEnabled": true
        },
        "persistentBrowser": {
            "mode": "never",
            "isEnabled": true
        }
    }
}
```

**Security Controls:**
- **Multi-Factor Authentication (MFA):** Required for all sensitive access
- **Device Compliance:** Device must be managed (Intune, Jamf), encrypted, and pass health attestation
- **Sign-in Frequency:** Forces re-authentication every 4 hours (reduces session hijacking risk)
- **Cloud App Security:** Real-time session monitoring to detect anomalous behavior even after initial authentication

---

## 12.3 Data Protection and Encryption Architecture

Data protection in the cloud spans encryption at rest (stored data), in transit (moving across networks), and in use (confidential computing). Proper key management is the foundation of cloud encryption.

### 12.3.1 Encryption at Rest

**Server-Side Encryption (SSE):**
Cloud providers automatically encrypt data at rest using AES-256. However, customers must choose key management strategies:

1. **Provider-Managed Keys (Default):** Provider controls key generation, rotation, and protection. Simple but offers less control.
2. **Customer-Managed Keys (CMK):** Customer creates keys in cloud KMS (Key Management Service) and controls rotation, deletion, and access policies. Required for many compliance regimes.
3. **Customer-Provided Keys (CSEK):** Customer provides their own keys; provider never sees the key material. Highest control, highest operational burden.
4. **Double Encryption:** Combine provider-managed encryption with application-level encryption for highly sensitive data.

**AWS KMS Key Policy Example:**

```json
{
    "Version": "2012-10-17",
    "Id": "key-default-1",
    "Statement": [
        {
            "Sid": "Enable IAM User Permissions",
            "Effect": "Allow",
            "Principal": {
                "AWS": "arn:aws:iam::123456789:root"
            },
            "Action": "kms:*",
            "Resource": "*"
        },
        {
            "Sid": "Allow Data Team to Use Key",
            "Effect": "Allow",
            "Principal": {
                "AWS": "arn:aws:iam::123456789:role/DataEngineeringRole"
            },
            "Action": [
                "kms:Encrypt",
                "kms:Decrypt",
                "kms:ReEncrypt*",
                "kms:GenerateDataKey*",
                "kms:DescribeKey"
            ],
            "Resource": "*",
            "Condition": {
                "StringEquals": {
                    "kms:ViaService": "s3.us-east-1.amazonaws.com",
                    "kms:CallerAccount": "123456789"
                },
                "Bool": {
                    "aws:MultiFactorAuthPresent": "true"
                }
            }
        },
        {
            "Sid": "Restrict Key Deletion",
            "Effect": "Deny",
            "Principal": {
                "AWS": "*"
            },
            "Action": "kms:ScheduleKeyDeletion",
            "Resource": "*",
            "Condition": {
                "Bool": {
                    "aws:MultiFactorAuthPresent": "false"
                }
            }
        },
        {
            "Sid": "Automatic Rotation",
            "Effect": "Allow",
            "Principal": {
                "AWS": "arn:aws:iam::123456789:root"
            },
            "Action": "kms:EnableKeyRotation",
            "Resource": "*"
        }
    ]
}
```

**Security Features:**
- **ViaService Condition:** Ensures key is only used through specific services (S3 in this case), preventing misuse through other APIs
- **MFA Enforcement:** Requires multi-factor authentication for cryptographic operations
- **Deletion Protection:** Prevents accidental or malicious key deletion without MFA

### 12.3.2 Encryption in Transit

All cloud traffic should use TLS 1.2 or higher. However, implementing proper certificate management and perfect forward secrecy requires attention.

**AWS ACM (Certificate Manager) with ALB (Application Load Balancer):**

```yaml
Resources:
  SSLCertificate:
    Type: AWS::CertificateManager::Certificate
    Properties:
      DomainName: app.company.com
      SubjectAlternativeNames:
        - api.company.com
      ValidationMethod: DNS
      DomainValidationOptions:
        - DomainName: app.company.com
          HostedZoneId: !Ref HostedZoneId
      KeyAlgorithm: RSA_2048
      Tags:
        - Key: Environment
          Value: Production

  SecureLoadBalancer:
    Type: AWS::ElasticLoadBalancingV2::LoadBalancer
    Properties:
      Type: application
      Scheme: internet-facing
      SecurityGroups:
        - !Ref ALBSecurityGroup
      Subnets: !Ref PublicSubnets

  HTTPSListener:
    Type: AWS::ElasticLoadBalancingV2::Listener
    Properties:
      LoadBalancerArn: !Ref SecureLoadBalancer
      Port: 443
      Protocol: HTTPS
      SslPolicy: ELBSecurityPolicy-TLS13-1-2-2021-06  # Forces TLS 1.2+
      Certificates:
        - CertificateArn: !Ref SSLCertificate
      DefaultActions:
        - Type: forward
          TargetGroupArn: !Ref WebTargetGroup

  # Security Policy: Redirect HTTP to HTTPS
  HTTPListener:
    Type: AWS::ElasticLoadBalancingV2::Listener
    Properties:
      LoadBalancerArn: !Ref SecureLoadBalancer
      Port: 80
      Protocol: HTTP
      DefaultActions:
        - Type: redirect
          RedirectConfig:
            Protocol: HTTPS
            Port: 443
            StatusCode: HTTP_301
```

**Best Practices:**
- **TLS 1.3:** Latest standard providing improved performance (0-RTT) and security
- **Certificate Rotation:** ACM automatically renews certificates; for imported certificates, implement 30-day expiration alerting
- **HSTS Headers:** Enforce HTTPS-only connections at the browser level

### 12.3.3 Confidential Computing

Confidential computing protects data while it is being processed (in memory) using hardware-based Trusted Execution Environments (TEEs). Cloud providers offer:
- **AWS Nitro Enclaves:** Isolated compute environments for processing sensitive data
- **Azure Confidential Computing:** Intel SGX-based VMs for encrypted in-use data
- **Google Confidential VMs:** AMD SEV-SNP based encryption of memory

**Use Cases:** Processing PII/PHI without exposing it to the host OS, secure multi-party computation, blockchain validation.

---

## 12.4 Network Security Architecture

Network security in the cloud moves beyond traditional perimeter-based firewalls to distributed, identity-aware controls.

### 12.4.1 Defense in Depth: Layered Security

**Layer 1: Edge Protection (DDoS and WAF)**
- **AWS Shield / Azure DDoS Protection:** Managed DDoS mitigation at the network and transport layers
- **Web Application Firewall (WAF):** Protects against OWASP Top 10 (SQL injection, XSS, CSRF)

**AWS WAF Rule Configuration:**

```json
{
  "Name": "ProductionWebACL",
  "DefaultAction": {
    "Block": {}  // Default deny - must explicitly allow
  },
  "Rules": [
    {
      "Name": "AWSManagedRulesCommonRuleSet",
      "Priority": 1,
      "Statement": {
        "ManagedRuleGroupStatement": {
          "VendorName": "AWS",
          "Name": "AWSManagedRulesCommonRuleSet",
          "RuleActionOverrides": [
            {
              "ActionToUse": {
                "Block": {}
              },
              "Name": "SizeRestrictions_BODY"
            }
          ]
        }
      },
      "OverrideAction": {
        "None": {}
      },
      "VisibilityConfig": {
        "SampledRequestsEnabled": true,
        "CloudWatchMetricsEnabled": true,
        "MetricName": "AWSManagedRulesCommonRuleSetMetric"
      }
    },
    {
      "Name": "RateLimitRule",
      "Priority": 2,
      "Statement": {
        "RateBasedStatement": {
          "Limit": 2000,  // Requests per 5 minutes per IP
          "AggregateKeyType": "IP",
          "ScopeDownStatement": {
            "NotStatement": {
              "Statement": {
                "IPSetReferenceStatement": {
                  "ARN": "arn:aws:wafv2:us-east-1:123456789:regional/ipset/TrustedIPs"
                }
              }
            }
          }
        }
      },
      "Action": {
        "Block": {}
      },
      "VisibilityConfig": {
        "SampledRequestsEnabled": true,
        "CloudWatchMetricsEnabled": true,
        "MetricName": "RateLimitRuleMetric"
      }
    },
    {
      "Name": "GeoBlockingRule",
      "Priority": 3,
      "Statement": {
        "GeoMatchStatement": {
          "CountryCodes": ["KP", "IR", "SY"],  // Sanctioned countries
          "ForwardedIPConfig": {
            "HeaderName": "X-Forwarded-For",
            "FallbackBehavior": "MATCH"
          }
        }
      },
      "Action": {
        "Block": {}
      },
      "VisibilityConfig": {
        "SampledRequestsEnabled": true,
        "CloudWatchMetricsEnabled": true,
        "MetricName": "GeoBlockingRuleMetric"
      }
    }
  ],
  "VisibilityConfig": {
    "SampledRequestsEnabled": true,
    "CloudWatchMetricsEnabled": true,
    "MetricName": "ProductionWebACL"
  }
}
```

**Layer 2: Network Segmentation (VPC/Subnet Design)**
- **Public Subnets:** Load balancers, bastion hosts only
- **Private Subnets:** Application workloads (no direct internet access)
- **Database Subnets:** Highly restricted, no internet gateway access
- **Transit Gateway:** Centralized routing between VPCs and on-premises

**Layer 3: Micro-Segmentation (Security Groups and NACLs)**
Security Groups are stateful (return traffic automatically allowed), while NACLs are stateless and provide subnet-level defense in depth.

```hcl
# Terraform: Defense in depth security groups
resource "aws_security_group" "app_tier" {
  name_prefix = "app-tier-"
  vpc_id      = aws_vpc.main.id
  description = "Application tier - restrictive ingress"

  # Only accept traffic from load balancer security group (not CIDR blocks)
  ingress {
    description     = "HTTPS from ALB only"
    from_port       = 443
    to_port         = 443
    protocol        = "tcp"
    security_groups = [aws_security_group.alb.id]  # Reference to ALB SG
  }

  egress {
    description = "HTTPS to database tier only"
    from_port   = 5432
    to_port     = 5432
    protocol    = "tcp"
    cidr_blocks = [aws_subnet.database.cidr_block]  # Only DB subnet
  }

  tags = {
    Name = "app-tier-sg"
  }
}

# NACL for additional subnet-level protection
resource "aws_network_acl" "database" {
  vpc_id     = aws_vpc.main.id
  subnet_ids = aws_subnet.database[*].id

  # Explicit deny of common attack ports at subnet level
  egress {
    protocol   = "tcp"
    rule_no    = 100
    action     = "deny"
    cidr_block = "0.0.0.0/0"
    from_port  = 22   # SSH
    to_port    = 22
  }

  egress {
    protocol   = "tcp"
    rule_no    = 200
    action     = "allow"
    cidr_block = aws_subnet.application.cidr_block
    from_port  = 5432
    to_port    = 5432
  }
}
```

### 12.4.2 Private Connectivity and Hybrid Networking

**AWS Transit Gateway / Azure Virtual WAN / Google Cloud Network Connectivity Center:**
Centralized hubs for connecting multiple VPCs/VNets, on-premises data centers, and SaaS providers without complex peering relationships.

**Site-to-Site VPN with BGP Routing:**

```hcl
# AWS Site-to-Site VPN with dynamic routing
resource "aws_customer_gateway" "main" {
  bgp_asn    = 65000
  ip_address = "203.0.113.1"  # On-premises VPN device IP
  type       = "ipsec.1"
  
  tags = {
    Name = "on-prem-gateway"
  }
}

resource "aws_vpn_gateway" "main" {
  vpc_id = aws_vpc.main.id
  
  tags = {
    Name = "cloud-vpn-gateway"
  }
}

resource "aws_vpn_connection" "main" {
  vpn_gateway_id      = aws_vpn_gateway.main.id
  customer_gateway_id = aws_customer_gateway.main.id
  type                = "ipsec.1"
  static_routes_only  = false  # Use BGP for dynamic route advertisement
  
  tunnel1_phase1_encryption_algorithms = ["AES256-GCM-16"]
  tunnel1_phase2_encryption_algorithms = ["AES256-GCM-16"]
  tunnel1_ike_versions                 = ["ikev2"]
  
  tunnel2_phase1_encryption_algorithms = ["AES256-GCM-16"]
  tunnel2_phase2_encryption_algorithms = ["AES256-GCM-16"]
  tunnel2_ike_versions                 = ["ikev2"]
}
```

**Security Benefits:**
- **BGP Route Propagation:** Automatically learns on-premises routes; failover between VPN tunnels without manual intervention
- **Encryption:** IKEv2 with AES256-GCM provides authenticated encryption with associated data (AEAD)
- **Segmentation:** Transit Gateway route tables isolate different environments (prod, dev, PCI) while sharing the same VPN connection

---

## 12.5 Compliance and Governance Automation

Security architectures must demonstrate compliance with frameworks (SOC 2, ISO 27001, PCI-DSS, HIPAA, GDPR) through automated evidence collection and continuous monitoring.

### 12.5.1 Compliance as Code

**AWS Config / Azure Policy / Google Organization Policy:**
Continuously monitor resource configurations against desired baselines and automatically remediate violations.

**Azure Policy Definition for Encryption Enforcement:**

```json
{
  "properties": {
    "displayName": "Require SQL Database Transparent Data Encryption",
    "description": "Ensures all SQL databases have TDE enabled for encryption at rest",
    "mode": "Indexed",
    "policyRule": {
      "if": {
        "allOf": [
          {
            "field": "type",
            "equals": "Microsoft.Sql/servers/databases"
          },
          {
            "field": "Microsoft.Sql/servers/databases/transparentDataEncryption.status",
            "notEquals": "Enabled"
          }
        ]
      },
      "then": {
        "effect": "DeployIfNotExists",
        "details": {
          "type": "Microsoft.Sql/servers/databases/transparentDataEncryption",
          "name": "current",
          "deploymentScope": "inner",
          "roleDefinitionIds": [
            "/providers/Microsoft.Authorization/roleDefinitions/9b7fa17d-e63e-47b0-bb0a-15c516ac86ec"
          ],
          "existenceCondition": {
            "field": "Microsoft.Sql/servers/databases/transparentDataEncryption.status",
            "equals": "Enabled"
          },
          "deployment": {
            "properties": {
              "mode": "incremental",
              "template": {
                "$schema": "https://schema.management.azure.com/schemas/2015-01-01/deploymentTemplate.json#",
                "contentVersion": "1.0.0.0",
                "resources": [
                  {
                    "name": "[concat(parameters('serverName'), '/', parameters('databaseName'), '/current')]",
                    "type": "Microsoft.Sql/servers/databases/transparentDataEncryption",
                    "apiVersion": "2014-04-01",
                    "properties": {
                      "status": "Enabled"
                    }
                  }
                ]
              },
              "parameters": {
                "serverName": {
                  "value": "[field('Microsoft.Sql/servers/databases/serverName')]"
                },
                "databaseName": {
                  "value": "[field('name')]"
                }
              }
            }
          }
        }
      }
    }
  }
}
```

**Automated Remediation:**
This policy automatically deploys TDE configuration to any SQL database created without encryption, ensuring continuous compliance without manual intervention.

### 12.5.2 Evidence Collection and Audit Trails

**AWS CloudTrail / Azure Activity Log / Google Cloud Audit Logs:**
Immutable logs of all API calls and administrative actions, centralized in SIEM (Security Information and Event Management) tools like Splunk, Sentinel, or Chronicle.

**Centralized Logging Architecture:**

```hcl
# Centralized audit logging across multiple AWS accounts
resource "aws_cloudtrail" "organization" {
  name           = "organization-audit-trail"
  s3_bucket_name = aws_s3_bucket.audit_logs.id
  is_multi_region_trail = true
  is_organization_trail = true  # Applies to all accounts in AWS Org
  
  enable_logging = true
  
  event_selector {
    read_write_type           = "All"
    include_management_events = true
    
    data_resource {
      type   = "AWS::S3::Object"
      values = ["arn:aws:s3:::"]
    }
  }
  
  insight_selector {
    insight_type = "ApiCallRateInsight"  # Detect unusual API activity
  }
  
  kms_key_id = aws_kms_key.audit_encryption.arn
  
  tags = {
    Purpose = "SecurityAudit"
    Compliance = "SOC2"
  }
}

# Log file validation (integrity checking)
resource "aws_cloudtrail" "validation" {
  enable_log_file_validation = true  # Digital signatures on logs
  # ... other configuration
}
```

**SIEM Integration:**
CloudWatch Logs / Azure Monitor Logs stream to SIEM for correlation rules:
- **Impossible Travel:** Same user logging in from New York and Tokyo within 5 minutes
- **Privilege Escalation:** IAM policy changes followed by sensitive resource access
- **Data Exfiltration:** Unusual S3 bucket access patterns or large data transfers

---

## 12.6 Chapter Summary and Transition

This chapter established the security foundations essential for cloud computing, beginning with the Shared Responsibility Model that defines the boundary between provider and customer obligations. We explored how these boundaries shift across IaaS, PaaS, and SaaS models, emphasizing that while providers secure the infrastructure, customers remain responsible for configuration, access management, and data classification.

We implemented Zero Trust architectures that assume breach, replacing implicit network trust with explicit verification of identity, device health, and contextual risk signals. Data protection strategies covered encryption at rest using customer-managed keys, encryption in transit with TLS 1.3, and emerging confidential computing for data in use. Network security evolved from perimeter-based models to defense-in-depth approaches utilizing WAFs, micro-segmentation, and private connectivity options like Transit Gateway and Site-to-Site VPN.

Finally, we examined compliance automation through policy-as-code frameworks that enforce encryption, logging, and configuration standards while automatically remediating violations, providing continuous evidence collection for audit frameworks.

While this chapter established the architectural security controls and frameworks, the effective implementation of these controls depends entirely on identity and access management. Encryption keys, network policies, and compliance rules all rely on robust identity verification and least-privilege access controls. Misconfigured IAM policies—overly permissive roles, lack of multi-factor authentication, or stale credentials—remain the leading cause of cloud security breaches, bypassing even the most sophisticated network segmentation and encryption.

In **Chapter 13: Identity and Access Management (IAM) Deep Dive**, we will focus specifically on the identity layer that underpins all cloud security. You will learn to implement sophisticated IAM architectures using Role-Based Access Control (RBAC) and Attribute-Based Access Control (ABAC), manage temporary credentials and service accounts securely, implement just-in-time access for privileged operations, and federate identities across hybrid and multi-cloud environments. We will explore cross-account access patterns, IAM policy optimization, and the tools for detecting and remediating identity-based risks at scale.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../4. advanced_cloud_architecture_and_patterns/11. hybrid_and_multi_cloud_architectures.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='13. identity_and_access_management_deep_dive.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
